In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer         
from sklearn.feature_extraction.text import TfidfVectorizer     
import gensim

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rajat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
unable to import 'smart_open.gcs', disabling that module


In [2]:
import time
import pickle

In [216]:
data = pd.read_csv("data/Reviews.csv")

In [217]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [218]:
data.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [219]:
data.shape

(568454, 10)

In [220]:
data = data[data['Score'] != 3]

In [221]:
data.Score.unique()

array([5, 1, 4, 2])

In [222]:
data['Score'].value_counts()

5    363122
4     80655
1     52268
2     29769
Name: Score, dtype: int64

In [223]:
def partition(x):
    if x < 3:
        return 'negative'
    return 'positive'

score = data['Score']
temp = score.map(partition)
data['Score'] = temp

In [224]:
data = data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"})

In [225]:
data = data[data['HelpfulnessNumerator'] <= data['HelpfulnessDenominator']]

In [226]:
data = data.iloc[:100000,:]

In [227]:
data_X = data['Text']
data_Y = data['Score']

In [228]:
data_Y.values[1]

'negative'

In [229]:
print(set(stopwords.words('english')))

{'being', 'through', 'now', 'where', 'be', 'very', "weren't", "don't", 'hadn', 'were', 'didn', 'for', 'yours', 'couldn', 'of', 'these', 'ourselves', 'at', 'mightn', 'myself', 'ours', 'him', 'by', "it's", 'haven', 'doesn', "wasn't", 'in', 'll', 'further', 'won', 'all', 'them', 'whom', 'had', 'and', 'my', 'when', 'those', 'into', 'nor', 'an', 'only', "should've", "shan't", 'am', 'own', 'between', 'their', 'i', 'same', 'so', 'm', 'y', 'does', 'itself', "shouldn't", "aren't", 'from', 'd', 'most', 'yourselves', 'has', 'did', 'such', 'weren', 'it', 'theirs', 'up', 'some', "mightn't", 'out', 'more', "isn't", "you're", 'who', 'wasn', 'after', 'we', 'against', 't', 'themselves', 'how', 'than', 're', 'her', 'can', 'any', "needn't", 'our', 'not', "mustn't", 'over', 'should', "that'll", 'just', 'isn', 'is', 'during', "doesn't", 'until', 'while', 'as', 'what', "she's", 'ma', 'about', "couldn't", 'under', "you'll", 'the', 'each', 'don', 'his', 'are', 've', 'shan', "you've", 'this', 's', 'was', "woul

In [12]:
import re
snow = nltk.stem.SnowballStemmer('english')
stop = set(stopwords.words('english'))

def cleanhtml(sentence):
    sentence = sentence.lower()                                       # Converting to lowercase
    cleanr = re.compile('<.*?>')
    sentence = re.sub(cleanr, ' ', sentence)                          # Cleaning Html tags
    
    return sentence

def cleanpunc(sentence):
    sentence = re.sub('[^A-Za-z0-9]+', '', sentence)                # Cleaning punctuations
    
    return sentence

In [231]:
temp = []
str1 = ''
positive_words = []
negative_words = []
i = 0
for sentence in data_X:
    words = []
    sentence = cleanhtml(sentence)
    for word in sentence.split():
        word = cleanpunc(word)
        if((word.isalpha()) & (len(word) > 2)): 
            if word not in stop:
                s = snow.stem(word).encode('utf8')
                words.append(s)
                if (data_Y.values)[i] == 'positive': 
                    positive_words.append(s) #list of all words used to describe positive reviews.
                if(data_Y.values)[i] == 'negative':
                    negative_words.append(s) #list of all words used to describe negative reviews.
            else:
                continue
        else:
                continue
    str1 = b" ".join(words)
    temp.append(str1)
    i += 1

In [261]:
temp[:10]

[b'bought sever vital can dog food product found good qualiti product look like stew process meat smell better labrador finicki appreci product better',
 b'product arriv label jumbo salt peanutsth peanut actual small size unsalt sure error vendor intend repres product jumbo',
 b'confect around centuri light pillowi citrus gelatin nut case filbert cut tini squar liber coat powder sugar tini mouth heaven chewi flavor high recommend yummi treat familiar stori lewi lion witch wardrob treat seduc edmund sell brother sister witch',
 b'look secret ingredi robitussin believ found got addit root beer extract order good made cherri soda flavor medicin',
 b'great taffi great price wide assort yummi taffi deliveri quick taffi lover deal',
 b'got wild hair taffi order five pound bag taffi enjoy mani flavor watermelon root beer melon peppermint grape etc complaint bit much redblack licoriceflavor piec particular favorit kid husband last two week would recommend brand taffi delight treat',
 b'saltwat

In [262]:
data_X = temp

In [263]:
df = pd.DataFrame(list(zip(data_X, data_Y)), columns = ['Text', 'Sentiment'] )

In [3]:
df.to_pickle('data/cleaned_file.pkl')

NameError: name 'df' is not defined

In [4]:
df = pd.read_pickle('data/cleaned_file.pkl')

In [5]:
df.head()

,Text,Sentiment
0,b'bought sever vital can dog food product foun...,positive
1,b'product arriv label jumbo salt peanutsth pea...,negative
2,b'confect around centuri light pillowi citrus ...,positive
3,b'look secret ingredi robitussin believ found ...,negative
4,b'great taffi great price wide assort yummi ta...,positive


In [6]:
df['Text'][2]

b'confect around centuri light pillowi citrus gelatin nut case filbert cut tini squar liber coat powder sugar tini mouth heaven chewi flavor high recommend yummi treat familiar stori lewi lion witch wardrob treat seduc edmund sell brother sister witch'

In [7]:
X = df["Text"]
y = df["Sentiment"]

In [8]:
df.shape, X.shape, y.shape

((100000, 2), (100000,), (100000,))

## Word2Vec 

In [9]:
from gensim.models import Word2Vec
import math

In [10]:
X_1 = X[:math.ceil(len(df)*0.8)]
y_1 = y[:math.ceil(len(df)*0.8)]
X_test = X[math.ceil(len(df)*0.8):]
y_test = y[math.ceil(len(df)*0.8):]

X_train = X_1[:math.ceil(len(df)*0.6)]
y_train = y_1[:math.ceil(len(df)*0.6)]
X_cv = X_1[math.ceil(len(df)*0.6):]
y_cv = y_1[math.ceil(len(df)*0.6):]

print(X_train.shape, y_train.shape, X_cv.shape, X_test.shape)

(60000,) (60000,) (20000,) (20000,)


In [14]:
list_of_sentence = []
for sentence in X_train:
    words = []
    sentence = sentence.decode('utf-8')
    sentence = cleanhtml(sentence)
    for word in sentence.split():
        word = cleanpunc(word)
        if(word.isalpha()):    
            words.append(word)
        else:
            continue 
    list_of_sentence.append(words)

In [15]:
print(list_of_sentence[:5])

[['bought', 'sever', 'vital', 'can', 'dog', 'food', 'product', 'found', 'good', 'qualiti', 'product', 'look', 'like', 'stew', 'process', 'meat', 'smell', 'better', 'labrador', 'finicki', 'appreci', 'product', 'better'], ['product', 'arriv', 'label', 'jumbo', 'salt', 'peanutsth', 'peanut', 'actual', 'small', 'size', 'unsalt', 'sure', 'error', 'vendor', 'intend', 'repres', 'product', 'jumbo'], ['confect', 'around', 'centuri', 'light', 'pillowi', 'citrus', 'gelatin', 'nut', 'case', 'filbert', 'cut', 'tini', 'squar', 'liber', 'coat', 'powder', 'sugar', 'tini', 'mouth', 'heaven', 'chewi', 'flavor', 'high', 'recommend', 'yummi', 'treat', 'familiar', 'stori', 'lewi', 'lion', 'witch', 'wardrob', 'treat', 'seduc', 'edmund', 'sell', 'brother', 'sister', 'witch'], ['look', 'secret', 'ingredi', 'robitussin', 'believ', 'found', 'got', 'addit', 'root', 'beer', 'extract', 'order', 'good', 'made', 'cherri', 'soda', 'flavor', 'medicin'], ['great', 'taffi', 'great', 'price', 'wide', 'assort', 'yummi', '

In [16]:
len(list_of_sentence)

60000

In [17]:
w2v_model = gensim.models.Word2Vec(list_of_sentence, min_count = 5, size = 50, workers = 4)

In [41]:
print(w2v_model.wv)

In [279]:
w2v_vocab = list(w2v_model.wv.vocab)

In [280]:
print(w2v_vocab[:50])

['bought', 'sever', 'vital', 'can', 'dog', 'food', 'product', 'found', 'good', 'qualiti', 'look', 'like', 'stew', 'process', 'meat', 'smell', 'better', 'labrador', 'finicki', 'appreci', 'arriv', 'label', 'jumbo', 'salt', 'peanut', 'actual', 'small', 'size', 'unsalt', 'sure', 'error', 'vendor', 'intend', 'repres', 'confect', 'around', 'centuri', 'light', 'citrus', 'gelatin', 'nut', 'case', 'cut', 'tini', 'squar', 'liber', 'coat', 'powder', 'sugar', 'mouth']


## Average Word2Vec 

In [281]:
from tqdm import tqdm

corpus_embedding = []
for sentence in tqdm(X_train):
#     print(sentence)
#     break
    sentence = sentence.decode('utf-8')
    sentence_embedding = np.zeros(50)
    count_words = 0
    for word in sentence.split():
        if word in w2v_vocab:
            embedding = w2v_model.wv[word]
            sentence_embedding += embedding
            count_words += 1
    if count_words != 0:
        sentence_embedding /= count_words
    corpus_embedding.append(sentence_embedding)

100%|██████████| 60000/60000 [01:32<00:00, 650.89it/s]


In [332]:
print(len(corpus_embedding))
print(len(corpus_embedding[0]))
corpus_embedding[:2]

60000
50


[array([ 0.61739673,  0.12484459, -0.06689609,  0.45843966,  0.21517037,
        -0.38145365, -0.22890107, -0.24621044,  0.18322493, -0.64715211,
         0.47312825,  0.1212746 , -0.20917033,  0.07304018,  0.1917164 ,
        -0.34190255, -0.0740689 ,  0.23553804, -0.19848122, -0.09747929,
        -0.40971219, -0.50984567,  0.7398974 , -0.17426051, -0.38910081,
         0.14594186, -0.15753686,  0.10157121, -0.83091147, -0.41507436,
        -0.23850454, -1.0695665 ,  0.02278422,  0.3257149 , -0.17303633,
        -0.6618879 , -0.17315514,  0.48021129,  0.80094977, -0.67133373,
         0.80095113, -0.01180873, -0.0398592 , -0.5492462 , -0.45810671,
         0.39854293,  1.05066226, -0.4883309 ,  0.86311581, -0.23029786]),
 array([-0.423879  ,  0.01845485, -0.5596409 , -0.84833058, -0.24737607,
         0.18218593,  0.25824744,  0.06347825, -0.52746832, -0.25563315,
        -0.04755123,  0.59254624, -0.33801098, -0.9102525 , -0.40943089,
         0.07774633, -0.1818896 ,  0.26879047, -0

In [283]:
corpus_embedding_test = []
for sentence in tqdm(X_test.values):
    sentence = sentence.decode('utf-8')
    sentence_embedding = np.zeros(50)
    count_words = 0
    for word in sentence.split():
        if word in w2v_vocab:
            embedding = w2v_model.wv[word]
            sentence_embedding += embedding
            count_words += 1
    if count_words != 0:
        sentence_embedding /= count_words
    corpus_embedding_test.append(sentence_embedding)

100%|██████████| 20000/20000 [00:30<00:00, 659.51it/s]


In [333]:
print(len(corpus_embedding_test))
print(len(corpus_embedding_test[0]))
corpus_embedding_test[:2]

20000
50


[array([ 0.34137867,  0.32644061, -0.6913592 ,  0.93042797, -0.24674746,
        -0.00793737,  0.83434894,  0.42846851, -0.46822411,  0.18145308,
         0.02695534,  0.28545126,  0.44953037,  0.20109814,  0.11302191,
         0.18967716, -1.05604081, -0.12670688,  0.56329733, -0.32371481,
         0.22560349,  0.31994825,  0.37654363, -0.04400121, -0.54762108,
         0.5262587 , -0.66112728, -0.25609161, -0.53637749,  0.06435166,
         0.03610195, -0.26324159, -0.07435891,  0.39390961, -0.28741728,
        -0.74851899,  0.38042809,  0.35705272, -0.76367375,  0.00314158,
         0.23122552, -0.44792692, -0.02893383, -0.63975109,  0.28802527,
        -0.17319469,  0.57268161,  0.14568441,  1.24386435, -0.01803521]),
 array([ 0.35155937,  0.13444979, -0.30987932,  0.76400572,  0.09771357,
        -0.0548918 ,  0.27184974,  0.26068094,  0.19945618,  0.07570894,
         0.44851215, -0.0939522 ,  0.27898915,  0.03982248,  0.00264986,
        -0.03002163, -0.94737078,  0.3978112 ,  0

In [331]:
corpus_embedding_cv = []
for sentence in tqdm(X_cv.values):
    sentence = sentence.decode('utf-8')
    sentence_embedding = np.zeros(50)
    count_words = 0
    for word in sentence.split():
        if word in w2v_vocab:
            embedding = w2v_model.wv[word]
            sentence_embedding += embedding
            count_words += 1
    if count_words != 0:
        sentence_embedding /= count_words
    corpus_embedding_cv.append(sentence_embedding)

100%|██████████| 20000/20000 [00:32<00:00, 619.80it/s]


In [334]:
print(len(corpus_embedding_cv))
print(len(corpus_embedding_cv[0]))
corpus_embedding_cv[:2]

20000
50


[array([-0.48616667,  0.18453874,  0.13360501,  0.49358281, -0.33790518,
         0.3935452 ,  0.17377513, -0.10749542, -0.43118972,  0.35650193,
        -0.17375508,  0.70026264, -1.51880368, -0.24281143,  0.64329223,
         0.41680116,  0.2482118 ,  0.10048615,  0.77865304, -0.35422269,
        -0.19091147,  0.72229682, -0.27655892, -0.6805847 , -1.58424106,
        -0.24681489,  0.01320604,  0.44208384,  0.38081266, -0.21585519,
        -0.71953296,  0.33142461, -1.55691555,  0.32053394,  0.39746836,
         0.6322432 ,  0.83148427,  1.34785175,  0.00430632, -0.13085715,
         0.45700192, -0.85015313,  0.16664648, -0.35655605,  0.39456441,
         0.35720955,  1.03568124, -1.00298616,  1.36411868,  0.12226187]),
 array([ 0.05629636,  0.13720081, -0.5643377 ,  0.73348429, -0.22710556,
         0.68583415,  0.7762212 ,  0.32816233, -0.58972414,  0.09781129,
         0.08961729,  0.71635393, -0.40924045,  0.55642931, -0.27097759,
         0.72200025,  0.2880372 ,  0.57930572,  0

In [335]:
X_train_DT = corpus_embedding
X_cv_DT = corpus_embedding_cv
X_test_DT = corpus_embedding_test

In [ ]:
myList = list(np.arange(1,11))

# empty list that will hold cv scores
cv_scores = []

for depth in myList:
    DT = DecisionTreeClassifier(max_depth = depth)
    scores = cross_val_score(DT, X_train, y_train, cv = 5, scoring = 'f1_weighted')
    cv_scores.append(scores.mean())
    
# changing to misclassification error
scr = [x for x in cv_scores]

optimal_depth = myList[scr.index(max(scr))]
print('\nThe optimal depth is %d.' % optimal_depth)

plt.plot(myList, scr)

plt.xlabel('Depth value')
plt.ylabel('f1-score')
plt.show()

In [289]:
X_train.values[:50]

array([b'bought sever vital can dog food product found good qualiti product look like stew process meat smell better labrador finicki appreci product better',
       b'product arriv label jumbo salt peanutsth peanut actual small size unsalt sure error vendor intend repres product jumbo',
       b'confect around centuri light pillowi citrus gelatin nut case filbert cut tini squar liber coat powder sugar tini mouth heaven chewi flavor high recommend yummi treat familiar stori lewi lion witch wardrob treat seduc edmund sell brother sister witch',
       b'look secret ingredi robitussin believ found got addit root beer extract order good made cherri soda flavor medicin',
       b'great taffi great price wide assort yummi taffi deliveri quick taffi lover deal',
       b'got wild hair taffi order five pound bag taffi enjoy mani flavor watermelon root beer melon peppermint grape etc complaint bit much redblack licoriceflavor piec particular favorit kid husband last two week would recommend br

## Average Word2Vec with tf_idf 

In [290]:
from sklearn.feature_extraction.text import TfidfVectorizer
model = TfidfVectorizer()

tf_idf_matrix = model.fit_transform(X_train.values)
dictionary = dict(zip(model.get_feature_names(), list(model.idf_)))

In [291]:
tfidf_embedding = []
for sentence in tqdm(X_train):
    sentence = sentence.decode('utf-8')
    sentence_embedding = np.zeros(50)
    weight_sum = 0
    for word in sentence.split():
        if word in w2v_vocab:
            embedding = w2v_model.wv[word]
            tf_idf = dictionary[word] * sentence.count(word)
            sentence_embedding += (embedding * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sentence_embedding /= weight_sum
    tfidf_embedding.append(sentence_embedding)

100%|██████████| 60000/60000 [01:49<00:00, 547.09it/s]


In [292]:
tfidf_embedding[:1]

[array([ 0.49783587,  0.12231475, -0.12560517,  0.36890145,  0.19909141,
        -0.40510795, -0.28291334, -0.18040224,  0.23266884, -0.47090244,
         0.4803518 ,  0.22619316, -0.29317434,  0.02209257,  0.08068267,
        -0.29437718,  0.05540543,  0.17686955, -0.13521102, -0.12169355,
        -0.36893002, -0.56393425,  0.63908845, -0.07714613, -0.30687754,
         0.08690704, -0.29011346,  0.18253315, -0.88322875, -0.49945644,
        -0.32704492, -1.01306613,  0.06844313,  0.36794265, -0.20144012,
        -0.54565518, -0.28555346,  0.37439441,  0.83892591, -0.52325959,
         0.60660107, -0.03196828,  0.01574486, -0.5170446 , -0.53218022,
         0.37927397,  1.00704537, -0.5859586 ,  0.8077709 ,  0.03277443])]

In [293]:
tfidf_embedding_test = []
for sentence in tqdm(X_test):
    sentence = sentence.decode('utf-8')
    sentence_embedding = np.zeros(50)
    weight_sum = 0
    for word in sentence.split():
        if word in w2v_vocab:
            embedding = w2v_model.wv[word]
            tf_idf = dictionary[word] * sentence.count(word)
            sentence_embedding += (embedding * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sentence_embedding /= weight_sum
    tfidf_embedding_test.append(sentence_embedding)

100%|██████████| 20000/20000 [00:35<00:00, 560.64it/s]


In [294]:
tfidf_embedding_test[:1]

[array([ 0.25171986,  0.23622187, -0.76169539,  0.68036085, -0.56270203,
        -0.207025  ,  0.99290651,  0.51128945, -0.49077345,  0.25647373,
        -0.42178422,  0.50763507,  0.57167984,  0.27064421, -0.01302953,
         0.15699311, -0.83758979, -0.00754108,  0.62960362, -0.47149934,
         0.07118557,  0.37806763,  0.48500558,  0.11890137, -0.56241435,
         0.38334651, -0.66468657, -0.2712106 , -0.32648738, -0.1279903 ,
        -0.07356892, -0.10431867, -0.01893302,  0.64409424, -0.27912134,
        -0.64605565,  0.39366062,  0.22463009, -0.43928205,  0.13364965,
         0.09099705, -0.50904631, -0.13208658, -0.49465468,  0.24093296,
        -0.13767485,  0.68397224,  0.13954011,  1.21340579, -0.01779093])]

In [295]:
tfidf_embedding_cv = []
for sentence in tqdm(X_cv):
    sentence = sentence.decode('utf-8')
    sentence_embedding = np.zeros(50)
    weight_sum = 0
    for word in sentence.split():
        if word in w2v_vocab:
            embedding = w2v_model.wv[word]
            tf_idf = dictionary[word] * sentence.count(word)
            sentence_embedding += (embedding * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sentence_embedding /= weight_sum
    tfidf_embedding_cv.append(sentence_embedding)

100%|██████████| 20000/20000 [00:38<00:00, 526.00it/s]


In [296]:
tfidf_embedding_cv[:1]

[array([-0.70055491,  0.23255103,  0.20840896,  0.22608537, -0.46473457,
         0.46686146, -0.09497256, -0.2967896 , -0.37357113,  0.55676659,
        -0.48692695,  0.94216948, -1.79095767, -0.53577616,  0.66806717,
         0.58600075,  0.4351291 ,  0.33036784,  0.95311044, -0.26563092,
        -0.35177702,  0.8849274 , -0.42681981, -0.70409937, -1.95830514,
        -0.46036565,  0.0285272 ,  0.62639581,  0.52375936, -0.27434186,
        -0.93943526,  0.33869912, -1.86096169,  0.3231114 ,  0.55746352,
         0.75543446,  0.95196995,  1.46532887,  0.19842415, -0.09156518,
         0.68453445, -1.08542465,  0.13417033, -0.12654752,  0.62075355,
         0.32465316,  1.17428756, -0.91031479,  1.5900295 ,  0.42382099])]

## bag of words

In [302]:
X = df["Text"]
y = df["Sentiment"]

In [303]:
df = df.iloc[:2000,:]

In [304]:
X_1 = X[:math.ceil(len(df)*.8)]
y_1 = y[:math.ceil(len(df)*.8)]
X_test = X[math.ceil(len(df)*.8):]
y_test = y[math.ceil(len(df)*.8):]

X_train = X_1[:math.ceil(len(df)*.6)]
y_train = y_1[:math.ceil(len(df)*.6)]
X_cv = X_1[math.ceil(len(df)*.6):]
y_cv = y_1[math.ceil(len(df)*.6):]

print(X_1.shape, y_1.shape, y_test.shape, X_train.shape, X_cv.shape)

(1600,) (1600,) (400,) (1200,) (400,)


In [305]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

count_vect = CountVectorizer() #in scikit-learn
X_train = count_vect.fit_transform(X_train)

In [306]:
X_test = count_vect.transform(X_test)
X_cv = count_vect.transform(X_cv)

In [307]:
X_train

<1200x4899 sparse matrix of type '<class 'numpy.int64'>'
	with 35259 stored elements in Compressed Sparse Row format>

## tf_idf

In [314]:
X_1 = X[:math.ceil(len(df)*.8)]
y_1 = y[:math.ceil(len(df)*.8)]
X_test = X[math.ceil(len(df)*.8):]
y_test = y[math.ceil(len(df)*.8):]

X_train = X_1[:math.ceil(len(df)*.6)]
y_train = y_1[:math.ceil(len(df)*.6)]
X_cv = X_1[math.ceil(len(df)*.6):]
y_cv = y_1[math.ceil(len(df)*.6):]

In [315]:
tf_idf_vect = TfidfVectorizer()
X_train = tf_idf_vect.fit_transform(X_train)

In [316]:
X_test = tf_idf_vect.transform(X_test)
X_cv = tf_idf_vect.transform(X_cv)

In [317]:
X_train

<1200x4899 sparse matrix of type '<class 'numpy.float64'>'
	with 35259 stored elements in Compressed Sparse Row format>